**Importing Necessary Libraries**

In [8]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

**Creating Floder Structure**

In [9]:
## Setup Paths
pos_path = os.path.join('Data', 'Positive')
neg_path = os.path.join('Data', 'Negative')
anc_path = os.path.join('Data', 'Anchor')

In [10]:
## Making the Directories
os.makedirs(pos_path)
os.makedirs(neg_path)
os.makedirs(anc_path)